In [2]:
import torch
DEVICE = torch.device('cuda')
import pandas as pd

In [50]:
data = pd.read_csv("./datasets/Product_Reviews_with_Description/reviewsWithSummary.csv")

In [2]:
data = pd.read_json("./datasets/Product_Reviews_with_Description/unique_product_reviews.json",orient='records',lines=True)
data = data.sample(5000)
data.reset_index(drop=True,inplace=True)

In [3]:
data.head()

,asin,title,description,reviewText,summary,overall,details,category
0,B017QKCC1Q,In The Kingdom&#39;s Name (Guardian of Scotla...,[],"[I love it ! The heroine Eva time travels, the...",[I love it! The heroine Eva time travels],[5],"{'File Size:': '2320 KB', 'Print Length:': '34...",Kindle_Store
1,B0140BL8G4,Craving HIM (Dominic Snow Book 3) eBook,[],"[AMAZING, AMAZING and super hot book!!! Domini...",[Super HOT!!!],[5],"{'File Size:': '1911 KB', 'Print Length:': '36...",Kindle_Store
2,B00M9CUJ6W,"AKG Vocal Condenser Microphone, Black, 6.00 x ...",[AKG's P220 is a large-diaphragm condenser mic...,[I've been using this microphone for about six...,[A GOOD INEXPENSIVE MICROPHONE],[4],{},Musical_Instruments
3,1572524421,Ulysses VHS,[VHS tape Ulysses],[I have seen this movie twice just want to hav...,[Just Another Good Classic Movie by Kirk Douglas],[5],None,Movies_and_TV
4,B00282L6T2,Tim-bor Professional Insecticide and Fungicide...,[It is a powder that is mixed with water and a...,[Easy to work with. Will have no idea if it wo...,[Easy to use],[5],{},Patio_Lawn_and_Garden


In [4]:
summaryData = pd.read_csv("./datasets/Reviews_with_Summary/Reviews.csv")

In [5]:
summaryData.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [52]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [7]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def cleanText(text):
    splitText = text.split()
    newText = []
    sw = stopwords.words('english')
    for word in splitText:
        if word in sw:
            continue
        if word in contractions:
            newText.append(contractions[word])
        else:
            newText.append(word)
    text = " ".join(newText)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)

    return text

def cleanTextList(textList):
    newText = []
    for text in textList:
        if text is None:
            continue
        newText.append(cleanText(text))
    return newText

summaryData['Text'] = summaryData['Text'].apply(cleanText)
data['reviewText'] = data['reviewText'].apply(cleanTextList)

[nltk_data] Downloading package stopwords to
[nltk_data]     /raid/home/akshat21515/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
data.head()

,asin,title,description,reviewText,summary,overall,details,category
0,B017QKCC1Q,In The Kingdom&#39;s Name (Guardian of Scotla...,[],[I love The heroine Eva time travels reader...,[I love it! The heroine Eva time travels],[5],"{'File Size:': '2320 KB', 'Print Length:': '34...",Kindle_Store
1,B0140BL8G4,Craving HIM (Dominic Snow Book 3) eBook,[],[AMAZING AMAZING super hot book Dominic Al...,[Super HOT!!!],[5],"{'File Size:': '1911 KB', 'Print Length:': '36...",Kindle_Store
2,B00M9CUJ6W,"AKG Vocal Condenser Microphone, Black, 6.00 x ...",[AKG's P220 is a large-diaphragm condenser mic...,[I ve using microphone six months recording st...,[A GOOD INEXPENSIVE MICROPHONE],[4],{},Musical_Instruments
3,1572524421,Ulysses VHS,[VHS tape Ulysses],[I seen movie twice want DVD I personally like...,[Just Another Good Classic Movie by Kirk Douglas],[5],None,Movies_and_TV
4,B00282L6T2,Tim-bor Professional Insecticide and Fungicide...,[It is a powder that is mixed with water and a...,[Easy work with Will idea works long time ho...,[Easy to use],[5],{},Patio_Lawn_and_Garden


In [9]:
summaryData.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I bought several Vitality canned dog food prod...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled Jumbo Salted Peanuts ...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This confection around centuries It light pi...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If looking secret ingredient Robitussin I beli...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy great price There wide assortment...


In [10]:
maxlen,avglen = 0,0
for sample in data['reviewText']:
    sample = " ".join(sample)
    maxlen = max(maxlen,len(sample))
    avglen += len(sample)
maxlen,avglen/data.shape[0]

(35577, 343.216)

In [11]:
data.shape

(5000, 8)

In [20]:
from transformers import BartTokenizer,BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained("kabita-choudhary/finetuned-bart-for-conversation-summary")
model = BartForConditionalGeneration.from_pretrained("kabita-choudhary/finetuned-bart-for-conversation-summary")
model.to(DEVICE)

def generateReviewSummary(text):
    text = " ".join(text)
    inps = tokenizer(text,return_tensors="pt",max_length=400,truncation=True,padding="max_length")
    inps.to(DEVICE)
    summaryIDs = model.generate(inps["input_ids"],num_beams=3,min_length=30,max_length=40,early_stopping=True)
    summary = tokenizer.decode(summaryIDs[0],skip_special_tokens=True)
    return summary

/raid/home/akshat21515/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-26 11:06:32.295380: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-26 11:06:33.163859: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [14]:
data['reviewSummary'] = data['reviewText'].map(generateReviewSummary)

In [15]:
data.to_csv("./datasets/Product_Reviews_with_Description/reviewsWithSummary.csv")

In [16]:
data.head()

,asin,title,description,reviewText,summary,overall,details,category,reviewSummary
0,B017QKCC1Q,In The Kingdom&#39;s Name (Guardian of Scotla...,[],[I love The heroine Eva time travels reader...,[I love it! The heroine Eva time travels],[5],"{'File Size:': '2320 KB', 'Print Length:': '34...",Kindle_Store,Eva time travels and meets William Wallace in ...
1,B0140BL8G4,Craving HIM (Dominic Snow Book 3) eBook,[],[AMAZING AMAZING super hot book Dominic Al...,[Super HOT!!!],[5],"{'File Size:': '1911 KB', 'Print Length:': '36...",Kindle_Store,Dominic Aleena's story captivated me from page...
2,B00M9CUJ6W,"AKG Vocal Condenser Microphone, Black, 6.00 x ...",[AKG's P220 is a large-diaphragm condenser mic...,[I ve using microphone six months recording st...,[A GOOD INEXPENSIVE MICROPHONE],[4],{},Musical_Instruments,I used a microphone six months ago in a record...
3,1572524421,Ulysses VHS,[VHS tape Ulysses],[I seen movie twice want DVD I personally like...,[Just Another Good Classic Movie by Kirk Douglas],[5],None,Movies_and_TV,I've seen the movie twice and I want to get th...
4,B00282L6T2,Tim-bor Professional Insecticide and Fungicide...,[It is a powder that is mixed with water and a...,[Easy work with Will idea works long time ho...,[Easy to use],[5],{},Patio_Lawn_and_Garden,"Easy work with Will idea works long time, howe..."


In [55]:
import re

def cleanCategory(x):
    return " ".join(x.split("_"))

def cleanText(text):
    text = str(text)
    splitText = text.split()
    newText = []
    sw = stopwords.words('english')
    for word in splitText:
        if word in sw:
            continue
        if word in contractions:
            newText.append(contractions[word])
        else:
            newText.append(word)
    text = " ".join(newText)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)

    return text

data['category'] = data['category'].apply(cleanCategory)
data['description'] = data['description'].apply(cleanText)

In [58]:
data['overall'] = data['overall'].apply(cleanText)

In [56]:
data['category'] = data['category'] + " [SEP] " + data['title']
data = data.drop(columns=['title'])

In [63]:
data

,Unnamed: 0,asin,description,reviewText,summary,overall,details,category,reviewSummary
0,0,B017QKCC1Q,,['I love The heroine Eva time travels reade...,['I love it! The heroine Eva time travels'],5,"{'File Size:': '2320 KB', 'Print Length:': '34...",Kindle Store [SEP] In The Kingdom&#39;s Name ...,Eva time travels and meets William Wallace in ...
1,1,B0140BL8G4,,['AMAZING AMAZING super hot book Dominic A...,['Super HOT!!!'],5,"{'File Size:': '1911 KB', 'Print Length:': '36...",Kindle Store [SEP] Craving HIM (Dominic Snow ...,Dominic Aleena's story captivated me from page...
2,2,B00M9CUJ6W,AKG s P220 large diaphragm condenser mic del...,['I ve using microphone six months recording s...,['A GOOD INEXPENSIVE MICROPHONE'],4,{},Musical Instruments [SEP] AKG Vocal Condenser ...,I used a microphone six months ago in a record...
3,3,1572524421,VHS tape Ulysses,['I seen movie twice want DVD I personally lik...,['Just Another Good Classic Movie by Kirk Doug...,5,NaN,Movies and TV [SEP] Ulysses VHS,I've seen the movie twice and I want to get th...
4,4,B00282L6T2,It powder mixed water applied wood structure...,['Easy work with Will idea works long time h...,['Easy to use'],5,{},Patio Lawn and Garden [SEP] Tim-bor Profession...,"Easy work with Will idea works long time, howe..."
...,...,...,...,...,...,...,...,...,...
4995,4995,B007FLN0OU,<B>The benefits using Sling carrier< B><BR>C...,['Used every day '],['Five Stars'],5,{'\n Product Dimensions: \n ': '9.2 x 2....,Pet Supplies [SEP] Alfie Pet Petoga Couture - ...,"Used every day, every single day. . - (e..."
4996,4996,6304458975,Clint Eastwood portrays Whitney directs fine...,['Great Video'],['Five Stars'],5,"{'Language:': 'English', 'ASIN: ': '6304458975'}",Movies and TV [SEP] Absolute Power VHS,Great Video. ... - . . - -. - - -
4997,4997,B000HCTY1W,Item 62929D Office Small Business 2007 c...,['I used this This Office suite I prefer The...,"[""The later editions aren't as good. I hated o...",5,{'\n Product Dimensions: \n ': '7.4 x 5....,Software [SEP] Microsoft Office Small Business...,I prefer the later editions of the Office suit...
4998,4998,B006Z63KBY,The black decker tl10 8 3 amp front tine til...,['Has done advertised do Was easy assemble r...,['Just as advertised'],5,{},Patio Lawn and Garden [SEP] Black &amp; Decker...,The sand box was easy to assemble. The sand bo...


In [67]:
data = data[~data['description'].str.contains(r"^\s+$")]

In [68]:
data

,Unnamed: 0,asin,description,reviewText,summary,overall,details,category,reviewSummary
2,2,B00M9CUJ6W,AKG s P220 large diaphragm condenser mic del...,['I ve using microphone six months recording s...,['A GOOD INEXPENSIVE MICROPHONE'],4,{},Musical Instruments [SEP] AKG Vocal Condenser ...,I used a microphone six months ago in a record...
3,3,1572524421,VHS tape Ulysses,['I seen movie twice want DVD I personally lik...,['Just Another Good Classic Movie by Kirk Doug...,5,NaN,Movies and TV [SEP] Ulysses VHS,I've seen the movie twice and I want to get th...
4,4,B00282L6T2,It powder mixed water applied wood structure...,['Easy work with Will idea works long time h...,['Easy to use'],5,{},Patio Lawn and Garden [SEP] Tim-bor Profession...,"Easy work with Will idea works long time, howe..."
5,5,B004YJZ5PI,THIS IS NOT A PERFECT GETAWAY To move four ...,['A lot 4 5 stars why This cheesy far fetc...,['Not a good movie'],2,"{'Language:': 'English', 'Subtitles:': 'Englis...",Movies and TV [SEP] Transit,A lot of people don't like this cheesy far fet...
6,6,B015NXVJY4,Start learning English instantly fun engagi...,['I reviewed nbsp <a data hook product link l...,['Instant Immersion vs Rosetta Stone - an actu...,3 5 5 5,{'\n Product Dimensions: \n ': '5.5 x 1....,Software [SEP] Learn English: Instant Immersio...,I reviewed nbsp <a data hook product link lin...
...,...,...,...,...,...,...,...,...,...
4995,4995,B007FLN0OU,<B>The benefits using Sling carrier< B><BR>C...,['Used every day '],['Five Stars'],5,{'\n Product Dimensions: \n ': '9.2 x 2....,Pet Supplies [SEP] Alfie Pet Petoga Couture - ...,"Used every day, every single day. . - (e..."
4996,4996,6304458975,Clint Eastwood portrays Whitney directs fine...,['Great Video'],['Five Stars'],5,"{'Language:': 'English', 'ASIN: ': '6304458975'}",Movies and TV [SEP] Absolute Power VHS,Great Video. ... - . . - -. - - -
4997,4997,B000HCTY1W,Item 62929D Office Small Business 2007 c...,['I used this This Office suite I prefer The...,"[""The later editions aren't as good. I hated o...",5,{'\n Product Dimensions: \n ': '7.4 x 5....,Software [SEP] Microsoft Office Small Business...,I prefer the later editions of the Office suit...
4998,4998,B006Z63KBY,The black decker tl10 8 3 amp front tine til...,['Has done advertised do Was easy assemble r...,['Just as advertised'],5,{},Patio Lawn and Garden [SEP] Black &amp; Decker...,The sand box was easy to assemble. The sand bo...


In [70]:
data.rename(columns={"reviewText":"text","overall":"rating"},inplace=True)
data['label'] = 0
data

/tmp/ipykernel_3695294/4227130147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={"reviewText":"text","overall":"rating"},inplace=True)
/tmp/ipykernel_3695294/4227130147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'] = 0


,Unnamed: 0,asin,description,text,summary,rating,details,category,reviewSummary,label
2,2,B00M9CUJ6W,AKG s P220 large diaphragm condenser mic del...,['I ve using microphone six months recording s...,['A GOOD INEXPENSIVE MICROPHONE'],4,{},Musical Instruments [SEP] AKG Vocal Condenser ...,I used a microphone six months ago in a record...,0
3,3,1572524421,VHS tape Ulysses,['I seen movie twice want DVD I personally lik...,['Just Another Good Classic Movie by Kirk Doug...,5,NaN,Movies and TV [SEP] Ulysses VHS,I've seen the movie twice and I want to get th...,0
4,4,B00282L6T2,It powder mixed water applied wood structure...,['Easy work with Will idea works long time h...,['Easy to use'],5,{},Patio Lawn and Garden [SEP] Tim-bor Profession...,"Easy work with Will idea works long time, howe...",0
5,5,B004YJZ5PI,THIS IS NOT A PERFECT GETAWAY To move four ...,['A lot 4 5 stars why This cheesy far fetc...,['Not a good movie'],2,"{'Language:': 'English', 'Subtitles:': 'Englis...",Movies and TV [SEP] Transit,A lot of people don't like this cheesy far fet...,0
6,6,B015NXVJY4,Start learning English instantly fun engagi...,['I reviewed nbsp <a data hook product link l...,['Instant Immersion vs Rosetta Stone - an actu...,3 5 5 5,{'\n Product Dimensions: \n ': '5.5 x 1....,Software [SEP] Learn English: Instant Immersio...,I reviewed nbsp <a data hook product link lin...,0
...,...,...,...,...,...,...,...,...,...,...
4995,4995,B007FLN0OU,<B>The benefits using Sling carrier< B><BR>C...,['Used every day '],['Five Stars'],5,{'\n Product Dimensions: \n ': '9.2 x 2....,Pet Supplies [SEP] Alfie Pet Petoga Couture - ...,"Used every day, every single day. . - (e...",0
4996,4996,6304458975,Clint Eastwood portrays Whitney directs fine...,['Great Video'],['Five Stars'],5,"{'Language:': 'English', 'ASIN: ': '6304458975'}",Movies and TV [SEP] Absolute Power VHS,Great Video. ... - . . - -. - - -,0
4997,4997,B000HCTY1W,Item 62929D Office Small Business 2007 c...,['I used this This Office suite I prefer The...,"[""The later editions aren't as good. I hated o...",5,{'\n Product Dimensions: \n ': '7.4 x 5....,Software [SEP] Microsoft Office Small Business...,I prefer the later editions of the Office suit...,0
4998,4998,B006Z63KBY,The black decker tl10 8 3 amp front tine til...,['Has done advertised do Was easy assemble r...,['Just as advertised'],5,{},Patio Lawn and Garden [SEP] Black &amp; Decker...,The sand box was easy to assemble. The sand bo...,0


In [103]:
data2 = data.drop_duplicates()
data2.rename(columns={'description':'prodDesc'},inplace=True)
data2

,Unnamed: 0,asin,prodDesc,text,summary,rating,details,category,reviewSummary,label
2,2,B00M9CUJ6W,AKG s P220 large diaphragm condenser mic del...,['I ve using microphone six months recording s...,['A GOOD INEXPENSIVE MICROPHONE'],4,{},Musical Instruments [SEP] AKG Vocal Condenser ...,I used a microphone six months ago in a record...,0
3,3,1572524421,VHS tape Ulysses,['I seen movie twice want DVD I personally lik...,['Just Another Good Classic Movie by Kirk Doug...,5,NaN,Movies and TV [SEP] Ulysses VHS,I've seen the movie twice and I want to get th...,0
4,4,B00282L6T2,It powder mixed water applied wood structure...,['Easy work with Will idea works long time h...,['Easy to use'],5,{},Patio Lawn and Garden [SEP] Tim-bor Profession...,"Easy work with Will idea works long time, howe...",0
5,5,B004YJZ5PI,THIS IS NOT A PERFECT GETAWAY To move four ...,['A lot 4 5 stars why This cheesy far fetc...,['Not a good movie'],2,"{'Language:': 'English', 'Subtitles:': 'Englis...",Movies and TV [SEP] Transit,A lot of people don't like this cheesy far fet...,0
6,6,B015NXVJY4,Start learning English instantly fun engagi...,['I reviewed nbsp <a data hook product link l...,['Instant Immersion vs Rosetta Stone - an actu...,3 5 5 5,{'\n Product Dimensions: \n ': '5.5 x 1....,Software [SEP] Learn English: Instant Immersio...,I reviewed nbsp <a data hook product link lin...,0
...,...,...,...,...,...,...,...,...,...,...
4995,4995,B007FLN0OU,<B>The benefits using Sling carrier< B><BR>C...,['Used every day '],['Five Stars'],5,{'\n Product Dimensions: \n ': '9.2 x 2....,Pet Supplies [SEP] Alfie Pet Petoga Couture - ...,"Used every day, every single day. . - (e...",0
4996,4996,6304458975,Clint Eastwood portrays Whitney directs fine...,['Great Video'],['Five Stars'],5,"{'Language:': 'English', 'ASIN: ': '6304458975'}",Movies and TV [SEP] Absolute Power VHS,Great Video. ... - . . - -. - - -,0
4997,4997,B000HCTY1W,Item 62929D Office Small Business 2007 c...,['I used this This Office suite I prefer The...,"[""The later editions aren't as good. I hated o...",5,{'\n Product Dimensions: \n ': '7.4 x 5....,Software [SEP] Microsoft Office Small Business...,I prefer the later editions of the Office suit...,0
4998,4998,B006Z63KBY,The black decker tl10 8 3 amp front tine til...,['Has done advertised do Was easy assemble r...,['Just as advertised'],5,{},Patio Lawn and Garden [SEP] Black &amp; Decker...,The sand box was easy to assemble. The sand bo...,0


In [74]:
from torch import Tensor
from torch.nn import Module,Dropout,Linear,LayerNorm
from torch.utils.data import DataLoader, Dataset

In [75]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

class CategoryReviewDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.stopwords = stopwords.words('english')
        keep = ['not','no','because','but','against','nor','very']
        for elem in keep:
            self.stopwords.remove(elem)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        txt = str(row["rating"]) + " " + row["text"]
        review = [word for word in txt.split() if word.lower() not in self.stopwords]
        label = row["label"]
        category = row["category"]
        
        reviewEncoding = {}
        reviewEncoding['input_ids'] = review
        if len(reviewEncoding['input_ids'])<511:
            reviewEncoding['input_ids'].append("[EOS]")
            reviewEncoding['input_ids'] += ["[PAD]"]*(511-len(reviewEncoding['input_ids']))
        else:
            reviewEncoding['input_ids'] = reviewEncoding['input_ids'][:510]
            reviewEncoding['input_ids'].append("[EOS]")
        reviewEncoding['category'] = category.split()
        if len(reviewEncoding['category'])<82:
            reviewEncoding['category'] += ["[PAD]"]*(82-len(reviewEncoding['category']))
        else:
            reviewEncoding['category'] = reviewEncoding['category'][:82]
        reviewEncoding["label"] = torch.tensor(label)
        return reviewEncoding

[nltk_data] Downloading package stopwords to
[nltk_data]     /raid/home/akshat21515/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [76]:
def collate_fn(batch):
    text, categories, labels = [], [], []
    for sample in batch:
        text.append(sample['input_ids'])
        categories.append(sample['category'])
        labels.append(sample["label"])

    return text, categories, torch.stack(labels)

In [77]:
class PositionalEncodings(Module):
    def __init__(self, d_model:int, max_len:int=512):
        super().__init__()
        self.d_model = d_model
        self.max_len = max_len

        pe = torch.zeros((max_len, d_model))
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x:Tensor):
        return (self.pe[:x.size(1), :]).unsqueeze(0)

In [78]:
from typing import Any,Tuple
import torch.nn.functional as F

class FakeReviewIdentifier(Module):
    def __init__(self,
                 model,
                 embeddingLayer,
                 embSize:int,
                 d_model:int,
                 vocabSize:int,
                 num_classes:int,
                 dropoutRate:float,
                 actOut:Any):
        
        super().__init__()
        self.embedding = embeddingLayer
        self.transform = Linear(embSize,d_model)
        self.positionalEncoding = PositionalEncodings(d_model)
        self.vocabSize = vocabSize

        self.model = model
        self.dropout = Dropout(dropoutRate)
        self.layerNorm = LayerNorm(d_model)

        self.fc1 = Linear(d_model,d_model//2)
        self.fc2 = Linear(d_model//2,d_model//4)
        self.fc3 = Linear(d_model//4,num_classes)

        self.act1 = F.tanh
        self.act2 = actOut

    def forward(self, text, category, attnMask):
        x = self.embedding(text)
        x = x.to(DEVICE)
        x = self.transform(x)
        
        xCategory = self.embedding(category)
        xCategory = xCategory.to(DEVICE)
        xCategory = self.transform(xCategory)
        
        x += self.positionalEncoding(x)
        xCategory += self.positionalEncoding(xCategory)

        xOut = self.model(x,xCategory,attnMask)
        xOut = xOut[:,0,:]
        xOut = xOut.to(torch.float32)
        
        out = self.fc1(xOut)
        out = self.fc2(out)
        out = self.act2(out)
        out = self.fc3(out)
        out = torch.softmax(out,dim=-1,dtype=torch.float32)

        return out

In [79]:
class CategoryAttention(Module):
    def __init__(self,
                 d_model:int=512,
                 n_heads:int=8):
        super().__init__()
        assert d_model % n_heads == 0, "d_model must be divisible by n_head"
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model//n_heads

        self.Wk = Linear(d_model,d_model)
        self.Wq = Linear(d_model,d_model)
        self.Wv = Linear(d_model,d_model)
        self.Wo = Linear(d_model,d_model)

    def forward(self,query,key,value,mask=None):
        batch_size = query.size(0)

        Q = self.Wq(query)
        K = self.Wk(key)
        V = self.Wv(value)

        Q = Q.view(batch_size,-1,self.n_heads,self.head_dim).permute(0,2,1,3)
        K = K.view(batch_size,-1,self.n_heads,self.head_dim).permute(0,2,1,3)
        V = V.view(batch_size,-1,self.n_heads,self.head_dim).permute(0,2,1,3)

        energy = torch.matmul(Q,K.permute(0,1,3,2)) / torch.sqrt(torch.tensor(self.head_dim,dtype=torch.float32))
        if mask is not None:
            energy.masked_fill_(mask==0,float("-1e20"))

        attnWeights = torch.softmax(energy,dim=1)
        out = torch.matmul(attnWeights,V)
        out = out.permute(0,2,1,3).contiguous().view(batch_size,-1,self.d_model)

        out = self.Wo(out)
        return out

In [80]:
class TransformerEncoderLayer(Module):
    def __init__(self,
                 d_model:int=512,
                 n_heads:int=8,
                 dim_feedforward:int=2048,
                 dropout:float=0.1,
                 activation:Any = F.relu_):
        
        super().__init__()
        self.selfAttn = CategoryAttention(d_model,n_heads)
        
        self.fc1 = Linear(d_model,dim_feedforward)
        self.dropout = Dropout(dropout)
        
        self.fc2 = Linear(dim_feedforward,d_model)
        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)

        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)
        self.dropout3 = Dropout(dropout)
        
        self.activation = activation

    def forward(self,
                sent1:Tensor,
                attnMask:Tensor=None,
                crossAttn:Tensor=None):
        
        attnSet = self.selfAttn(sent1,sent1,sent1,attnMask)
        resConn = self.dropout1(attnSet)
        if crossAttn is not None:
            resConn += self.dropout2(crossAttn)
        normOut = self.norm1(resConn)

        FFNOutp = self.activation(self.fc1(normOut))
        FFNProd = self.fc2(self.dropout(FFNOutp))

        out = resConn + self.dropout3(FFNProd)
        out = self.norm2(out)
        return out

In [81]:
from torch.nn import ModuleList

class TransformerEncoder(Module):
    def __init__(self,
                 encoder_layer:Module,
                 num_layers:int=8,
                 d_model:int=512,
                 n_heads:int=8,
                 dim_feedforward:int=2048,
                 dropout:float=0.1,
                 activation:Any=F.relu_):
        
        super().__init__()
        self.catAttn = CategoryAttention(d_model,n_heads)
        self.layers = ModuleList([encoder_layer(d_model,n_heads,dim_feedforward,dropout,activation) for _ in range(num_layers)])

    def forward(self,
                sent1:Tensor,
                sent2:Tensor,
                mask:Tuple[Tensor]=None):
        out = sent1
        attnCat = self.catAttn(sent1,sent2,sent2,mask[0])
        for layer in self.layers:
            out = layer(out,mask[1],attnCat)
        return out

In [82]:
from gensim.models.fasttext import load_facebook_vectors

def FasttextEmbedding(sentences,model=load_facebook_vectors("cc.en.300.bin")):
    embeds = []
    for sentence in sentences:
        wordEmbeds = []
        for i in range(len(sentence)):
            wordEmbeds.append(torch.tensor(model[sentence[i]]))
        if wordEmbeds:
            sentenceEmbedding = torch.stack(wordEmbeds)
            clsEmbed = torch.mean(sentenceEmbedding,dim=0).unsqueeze_(0)
            if len(sentences[0])==512:
                sentenceEmbedding = torch.cat([clsEmbed,sentenceEmbedding],dim=0)
            embeds.append(sentenceEmbedding)
    return torch.stack(embeds)

In [83]:
from torch.nn import CrossEntropyLoss
import torch.nn as nn

embSize = 300
d_model = 128
n_heads = 16
num_layers = 4
num_classes = 2
vocab_size = len(tokenizer)
dropout_rate = 0.25
dim_feedforward = 256
activation = F.relu_

baseModel = TransformerEncoder(
    TransformerEncoderLayer,
    num_layers,
    d_model,
    n_heads,
    dim_feedforward,
    dropout_rate,
    activation
)

fakeClassifier = FakeReviewIdentifier(baseModel,FasttextEmbedding,embSize,d_model,vocab_size,num_classes,dropout_rate,F.leaky_relu_)
checkpoint = torch.load("./checkpoints/FakeClassifier.pt")
fakeClassifier.load_state_dict(checkpoint.state_dict())
loss_fn = CrossEntropyLoss()
fakeClassifier = fakeClassifier.to(DEVICE)

In [84]:
def generateMask(t1,t2):
    attnMask = []
    for i in range(len(t1)):
        s1 = t1[i]
        curMask = []
        for word in s1:
            if word=="[PAD]":
                curMask.append(0)
            else:
                curMask.append(1)
        s1Mask = torch.tensor(curMask).unsqueeze_(1)

        s2 = t2[i]
        curMask = []
        for word in s2:
            if word=="[PAD]":
                curMask.append(0)
            else:
                curMask.append(1)
        s2Mask = torch.tensor(curMask).unsqueeze_(0)
        attnMask.append(torch.matmul(s1Mask,s2Mask))
    return torch.stack(attnMask).unsqueeze_(1)

In [85]:
dataLoader = DataLoader(CategoryReviewDataset(data2),batch_size=256,shuffle=True,collate_fn=collate_fn)

In [90]:
realReviews = []
realProds = []

for text,category,label in dataLoader:
    label = label.to(DEVICE)
    with torch.set_grad_enabled(False):
        out = fakeClassifier(text, category, attnMask=(generateMask(text,category).to(DEVICE),generateMask(text,text).to(DEVICE)))
        _,preds = torch.max(out,dim=1)
        
        for i in range(len(text)):
            if preds.tolist()[i]==0:
                realReviews.append([text[i]])
                realProds.append([category[i]])

In [91]:
len(realReviews),len(realProds)

(2726, 2726)

In [92]:
from sentence_transformers import SentenceTransformer,util

similarityModel = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [136]:
sample = data2[data2['asin']=='B00QXIIZ3W']
s1 = str(sample['reviewSummary'])
s2 = str(sample['prodDesc'])[9:-30]
s1,s2

('3030    Joaquin Phoenix\'s performance in "The Big Lebo...\nName: reviewSummary, dtype: object',
 ' <  CDATA     Inherent Vice  Blu ray     In L...')

In [137]:
sents = [s1,s2]
sentEmbs = similarityModel.encode(sents,convert_to_tensor=True)

In [138]:
cos_sim_matrix = util.pytorch_cos_sim(sentEmbs, sentEmbs)
cos_sim_matrix

tensor([[1.0000, 0.1493],
        [0.1493, 1.0000]], device='cuda:0')